# Preamble

In [1]:
import os, sys
import pandas as pd
import numpy as np

In [2]:
sys.path.append('/Users/lindenmp/Dropbox/Work/ResProjects/NormativeNeuroDev_CrossSec_DWI/code/func/')
from proj_environment import set_proj_env
from func import get_synth_cov

In [3]:
train_test_str = 'squeakycleanExclude'
exclude_str = 't1Exclude'
parc_str = 'schaefer'
parc_scale = 200
extra_str = ''
# extra_str = '_nuis-netdens'
# extra_str = '_nuis-str'
parcel_names, parcel_loc, drop_parcels, num_parcels, yeo_idx, yeo_labels = set_proj_env(train_test_str = train_test_str, exclude_str = exclude_str,
                                                                            parc_str = parc_str, parc_scale = parc_scale, extra_str = extra_str)

In [4]:
print(os.environ['MODELDIR_BASE'])
print(os.environ['MODELDIR'])

/Users/lindenmp/Dropbox/Work/ResProjects/NormativeNeuroDev_CrossSec_DWI/analysis/normative/t1Exclude/squeakycleanExclude/schaefer_200_streamlineCount
/Users/lindenmp/Dropbox/Work/ResProjects/NormativeNeuroDev_CrossSec_DWI/analysis/normative/t1Exclude/squeakycleanExclude/schaefer_200_streamlineCount


## Load data

In [5]:
# Load data
df = pd.read_csv(os.path.join(os.environ['MODELDIR_BASE'], 'df_pheno.csv'))
df.set_index(['bblid', 'scanid'], inplace = True)

df_node = pd.read_csv(os.path.join(os.environ['MODELDIR'], 'df_node_clean.csv'))
df_node.set_index(['bblid', 'scanid'], inplace = True)

# adjust sex to 0 and 1
df['sex_adj'] = df.sex - 1
print(df.shape)
print(df_node.shape)

(1068, 47)
(1068, 1001)


In [6]:
df.head()

,,squeakycleanExclude,ageAtScan1,ageAtScan1_Years,sex,race2,handednessv2,dti64MeanAbsRMS,dti64MeanRelRMS,dti64MaxAbsRMS,dti64MaxRelRMS,...,F1_Social_Cognition_Efficiency_Ar,F2_Complex_Reasoning_Efficiency_Ar,F3_Memory_Efficiency_Ar,F4_Executive_Efficiency_Ar,F1_Slow_Speed_Ar,F2_Memory_Speed_Ar,F3_Fast_Speed_Ar,streamline_count,network_density,sex_adj
bblid,scanid,,,,,,,,,,,,,,,,,,,,,
81287,2738,0,240,20.0,2,1,1,0.351665,0.154478,0.557358,0.272518,...,1.057700,1.601508,0.830148,0.514279,0.391313,-0.105869,-0.156787,272317.0,0.121457,1
80680,2739,1,253,21.1,1,1,1,0.531246,0.208078,0.754717,0.354560,...,0.595196,1.647967,0.379309,-0.389853,0.439099,0.709515,0.355577,257610.0,0.113166,0
81754,2740,1,232,19.3,2,1,1,0.310943,0.218462,0.460633,0.387235,...,0.307979,1.134216,0.677509,1.006161,1.177551,0.935206,1.335039,249019.0,0.128593,1
81903,2749,0,231,19.2,2,1,1,0.432610,0.283153,0.819576,0.509537,...,1.324297,1.886781,1.634455,0.527513,0.783375,1.272440,0.333475,208434.0,0.121055,1
81043,2750,0,249,20.8,2,2,1,0.162409,0.096761,0.429102,0.259678,...,0.759529,0.261837,0.306490,0.001343,0.748104,1.059229,0.370116,246636.0,0.127136,1


In [7]:
df_node.head()

,,squeakycleanExclude,ct_0,ct_1,ct_2,ct_3,ct_4,ct_5,ct_6,ct_7,ct_8,...,mc_190,mc_191,mc_192,mc_193,mc_194,mc_195,mc_196,mc_197,mc_198,mc_199
bblid,scanid,,,,,,,,,,,,,,,,,,,,,
81287,2738,0,-0.167822,-0.194174,-0.740807,-0.662708,-0.509090,-0.943489,-0.773872,-1.045438,-1.006088,...,0.037584,-0.028900,0.012144,0.052015,0.012362,0.016552,0.000020,0.000590,0.004916,0.005701
80680,2739,1,-0.193851,0.420116,0.252830,0.361275,-0.364400,-0.021903,0.342078,-0.431227,-0.390328,...,0.025546,0.000612,-0.077385,-0.034756,-0.007081,-0.007883,-0.006403,-0.002452,-0.010866,0.000855
81754,2740,1,0.388032,0.493616,0.894609,0.043516,-0.386255,-0.165124,0.709997,0.729123,0.325239,...,0.002266,0.058185,0.034433,0.011456,-0.007728,0.017353,0.022024,0.005748,0.005398,0.010287
81903,2749,0,-0.012320,-0.017624,-0.448293,0.154950,0.241234,0.118129,0.070863,-0.273430,0.124748,...,0.027558,0.055934,-0.008643,0.034269,0.001709,0.009815,0.011814,0.004840,0.004459,0.007920
81043,2750,0,-0.422012,-0.229684,-1.003900,0.223487,-0.181447,-0.176094,-0.197750,-0.255772,0.047815,...,-0.014156,0.051863,-0.030161,0.036038,0.006408,0.014444,0.015544,-0.000401,-0.002940,0.001917


# Prepare files for normative modelling

In [8]:
# Note, 'ageAtScan1_Years' is assumed to be covs[0] and 'sex_adj' is assumed to be covs[1]
# if more than 2 covs are to be used, append to the end and age/sex will be duplicated accordingly in the forward model
covs = ['ageAtScan1', 'sex_adj']

print(covs)
num_covs = len(covs)
print(num_covs)

['ageAtScan1', 'sex_adj']
2


In [9]:
extra_str_2 = ''

## Primary model (train/test split)

In [10]:
# Create subdirectory for specific normative model -- labeled according to parcellation/resolution choices and covariates
normativedir = os.path.join(os.environ['MODELDIR'], '+'.join(covs) + extra_str_2 + '/')
print(normativedir)
if not os.path.exists(normativedir): os.mkdir(normativedir);

/Users/lindenmp/Dropbox/Work/ResProjects/NormativeNeuroDev_CrossSec_DWI/analysis/normative/t1Exclude/squeakycleanExclude/schaefer_200_streamlineCount/ageAtScan1+sex_adj/


In [11]:
# Write out training -- retaining only residuals from nuissance regression
df[df[train_test_str] == 0].to_csv(os.path.join(normativedir, 'train.csv'))
df[df[train_test_str] == 0].to_csv(os.path.join(normativedir, 'cov_train.txt'), columns = covs, sep = ' ', index = False, header = False)

resp_train = df_node[df_node[train_test_str] == 0].drop(train_test_str, axis = 1)
mask = np.all(np.isnan(resp_train), axis = 1)
if np.any(mask): print("Warning: NaNs in response train")
resp_train.to_csv(os.path.join(normativedir, 'resp_train.csv'))
resp_train.to_csv(os.path.join(normativedir, 'resp_train.txt'), sep = ' ', index = False, header = False)

# Write out test -- retaining only residuals from nuissance regression
df[df[train_test_str] == 1].to_csv(os.path.join(normativedir, 'test.csv'))
df[df[train_test_str] == 1].to_csv(os.path.join(normativedir, 'cov_test.txt'), columns = covs, sep = ' ', index = False, header = False)

resp_test = df_node[df_node[train_test_str] == 1].drop(train_test_str, axis = 1)
mask = np.all(np.isnan(resp_test), axis = 1)
if np.any(mask): print("Warning: NaNs in response train")
resp_test.to_csv(os.path.join(normativedir, 'resp_test.csv'))
resp_test.to_csv(os.path.join(normativedir, 'resp_test.txt'), sep = ' ', index = False, header = False)

print(str(resp_train.shape[1]) + ' features written out for normative modeling')

1000 features written out for normative modeling


### Forward variants

In [12]:
fwddir = os.path.join(normativedir, 'forward/')
if not os.path.exists(fwddir): os.mkdir(fwddir)

# Synthetic cov data
x = get_synth_cov(df, cov = 'ageAtScan1', stp = 12)

if 'sex_adj' in covs:
    # Produce gender dummy variable for one repeat --> i.e., to account for two runs of ages, one per gender
    gender_synth = np.concatenate((np.ones(x.shape),np.zeros(x.shape)), axis = 0)

# concat
synth_cov = np.concatenate((np.matlib.repmat(x, 2, 1), np.matlib.repmat(gender_synth, 1, 1)), axis = 1)
print(synth_cov.shape)

# write out
np.savetxt(os.path.join(fwddir, 'synth_cov_test.txt'), synth_cov, delimiter = ' ', fmt = ['%.1f', '%.d'])

(30, 2)


### Cross-val variant

In [13]:
# Create subdirectory for specific normative model -- labeled according to parcellation/resolution choices and covariates
cvdir = os.path.join(normativedir, 'cv/')
if not os.path.exists(cvdir): os.mkdir(cvdir)